In [15]:
from dotenv import load_dotenv
import os
import pandas as pd
from llama_index.experimental.query_engine import PandasQueryEngine
from prompts import new_prompt, instruction_str, context
from note_engine import note_engine
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from pdf import canada_engine

load_dotenv()

True

In [16]:
population_path = os.path.join("data", "population.csv")
population_df = pd.read_csv(population_path)

population_query_engine = PandasQueryEngine(
    df=population_df, verbose=True, instruction_str=instruction_str
)
population_query_engine.update_prompts({"pandas_prompt": new_prompt})

tools = [
    note_engine,
    QueryEngineTool(
        query_engine=population_query_engine,
        metadata=ToolMetadata(
            name="population_data",
            description="this gives information at the world population and demographics",
        ),
    ),
    QueryEngineTool(
        query_engine=canada_engine,
        metadata=ToolMetadata(
            name="canada_data",
            description="this gives detailed information about canada the country",
        ),
    ),
]

In [14]:
llm = OpenAI(model="gpt-4o-mini")
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, context=context)

while (prompt := input("Enter a prompt (q to quit): ")) != "q":
    result = agent.query(prompt)
    print(result)


> Running step 1ea0fafa-2975-4f29-995b-cc831127ea4d. Step input: what is the population if India
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: population_data
Action Input: {'input': 'India'}
> Pandas Instructions:
```
df[df['Country'] == 'India']
```
> Pandas Output:     Rank Country  Population2023 YearlyChange  NetChange  Density(P/Km²)  \
95     1   India      1428627663       0.81 %   11454490             481   

    Land Area(Km²)  Migrants(net)  Fert.Rate  MedianAge UrbanPop% WorldShare  
95         2973190        -486136        2.0       28.0      36 %    17.76 %  
Observation:     Rank Country  Population2023 YearlyChange  NetChange  Density(P/Km²)  \
95     1   India      1428627663       0.81 %   11454490             481   

    Land Area(Km²)  Migrants(net)  Fert.Rate  MedianAge UrbanPop% WorldShare  
95         2973190        -486136        2.0       28.0      36 %    17.76 %  
> Running step 50a2d571-e5